# LLM-Powered ISO 8583 Features

This notebook demonstrates the AI-powered features in iso8583sim:
- **MessageExplainer**: Explain messages in plain English
- **MessageGenerator**: Generate messages from natural language

## Prerequisites

Install an LLM provider:
```bash
pip install iso8583sim[anthropic]  # Claude
pip install iso8583sim[openai]     # GPT
pip install iso8583sim[google]     # Gemini
pip install iso8583sim[ollama]     # Local (Llama/Mistral)
pip install iso8583sim[llm]        # All providers
```

Set your API key:
```bash
export ANTHROPIC_API_KEY="sk-ant-..."
# or
export OPENAI_API_KEY="sk-..."
# or
export GOOGLE_API_KEY="..."
```

In [ ]:
import sys

sys.path.insert(0, "..")

from iso8583sim.core.builder import ISO8583Builder
from iso8583sim.core.types import ISO8583Message
from iso8583sim.demo import generate_auth_request, generate_emv_auth, pretty_print

## Check Available Providers

First, let's see which LLM providers are available:

In [ ]:
from iso8583sim.llm import list_available_providers, list_installed_providers

print("Installed providers (packages installed):")
print(f"  {list_installed_providers() or 'None'}")

print("\nAvailable providers (installed + configured):")
print(f"  {list_available_providers() or 'None - set API key env var'}")

## MessageExplainer

The `MessageExplainer` takes an ISO 8583 message and provides a human-readable explanation.

In [ ]:
from iso8583sim.llm import MessageExplainer

# Create an explainer (auto-detects available provider)
try:
    explainer = MessageExplainer()
    print(f"Using provider: {explainer.provider.name}")
except Exception as e:
    print(f"No LLM provider available: {e}")
    explainer = None

### Explain a Message Object

In [ ]:
# Create a sample message
message = generate_auth_request(
    pan="4111111111111111",
    amount=10000,  # $100.00
    terminal_id="TERM0001",
    merchant_id="GASSTATION12345",
)

print("Message to explain:")
pretty_print(message)

if explainer:
    print("\n" + "=" * 60)
    print("LLM Explanation:")
    print("=" * 60)
    explanation = explainer.explain(message)
    print(explanation)

### Explain a Raw Message String

In [ ]:
builder = ISO8583Builder()
raw_message = builder.build(message)

print(f"Raw message: {raw_message[:80]}...")

if explainer:
    print("\nExplanation:")
    explanation = explainer.explain(raw_message)
    print(explanation)

### Explain EMV/Chip Card Message

In [ ]:
emv_message = generate_emv_auth(
    pan="4111111111111111",
    amount=25000,  # $250.00
    cryptogram="1234567890ABCDEF",
)

print("EMV Message:")
pretty_print(emv_message)

if explainer:
    print("\n" + "=" * 60)
    print("LLM Explanation of EMV Transaction:")
    print("=" * 60)
    explanation = explainer.explain(emv_message, verbose=True)
    print(explanation)

### Explain Specific Fields

In [ ]:
if explainer:
    # Explain response codes
    print("Explaining Response Code 51:")
    print(explainer.explain_response_code("51"))

    print("\n" + "=" * 40)

    # Explain a specific field
    print("\nExplaining Field 22 (POS Entry Mode):")
    print(explainer.explain_field(22, "051"))

## MessageGenerator

The `MessageGenerator` creates ISO 8583 messages from natural language descriptions.

In [ ]:
from iso8583sim.llm import MessageGenerator

try:
    generator = MessageGenerator()
    print(f"Using provider: {generator.provider.name}")
except Exception as e:
    print(f"No LLM provider available: {e}")
    generator = None

### Generate from Natural Language

In [ ]:
if generator:
    # Generate a simple purchase
    description = "$50 VISA purchase at a coffee shop"
    print(f"Description: {description}")
    print("\nGenerating message...")

    try:
        message = generator.generate(description, validate=False)
        print("\nGenerated Message:")
        pretty_print(message)
    except Exception as e:
        print(f"Generation failed: {e}")

In [ ]:
if generator:
    # Generate a refund
    description = "Refund $25 to Mastercard ending in 4444 at ACME Store"
    print(f"Description: {description}")
    print("\nGenerating message...")

    try:
        message = generator.generate(description, validate=False)
        print("\nGenerated Refund Message:")
        pretty_print(message)
    except Exception as e:
        print(f"Generation failed: {e}")

In [ ]:
if generator:
    # Generate a balance inquiry
    description = "Balance inquiry for VISA card at ATM"
    print(f"Description: {description}")
    print("\nGenerating message...")

    try:
        message = generator.generate(description, validate=False)
        print("\nGenerated Balance Inquiry:")
        pretty_print(message)
    except Exception as e:
        print(f"Generation failed: {e}")

### Suggest Missing Fields

In [ ]:
if generator:
    # Create a partial message
    partial_message = ISO8583Message(
        mti="0100",
        fields={
            0: "0100",
            2: "4111111111111111",
            4: "000000010000",
        },
    )

    print("Partial Message:")
    print(f"  MTI: {partial_message.mti}")
    print(f"  Fields: {list(partial_message.fields.keys())}")

    print("\nSuggesting missing fields...")
    suggestions = generator.suggest_fields(partial_message)

    if suggestions:
        print("\nSuggested Fields:")
        for field_num, value in sorted(suggestions.items()):
            print(f"  F{field_num:03d}: {value}")
    else:
        print("No suggestions available")

## Using Specific Providers

You can specify which LLM provider to use:

In [ ]:
# Use a specific provider by name
try:
    from iso8583sim.llm import get_provider

    # Try different providers
    for provider_name in ["anthropic", "openai", "google", "ollama"]:
        try:
            provider = get_provider(provider_name)
            print(f"✓ {provider_name}: {provider.model}")
        except Exception as e:
            print(f"✗ {provider_name}: {e}")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Use provider with custom settings
try:
    from iso8583sim.llm.providers.anthropic import AnthropicProvider

    # Use a specific model
    provider = AnthropicProvider(model="claude-3-haiku-20240307")
    explainer = MessageExplainer(provider=provider)

    print(f"Using: {explainer.provider.name} / {explainer.provider.model}")
except Exception as e:
    print(f"Could not create custom provider: {e}")

## Error Handling

The LLM module provides clear error messages:

In [ ]:
from iso8583sim.llm import LLMError, ProviderConfigError, ProviderNotAvailableError

# Example error handling
try:
    from iso8583sim.llm import get_provider

    provider = get_provider("invalid_provider")
except ProviderConfigError as e:
    print(f"Config Error: {e}")
except ProviderNotAvailableError as e:
    print(f"Not Available: {e}")
except LLMError as e:
    print(f"LLM Error: {e}")

## Next Steps

- Try different LLM providers to see how they compare
- Use the generator to create test message datasets
- Combine with validation to ensure generated messages are correct
- Check out the API reference for more advanced usage